### Multiprocessing - test

In [ ]:
import multiprocessing

print("Number of cpu : ", multiprocessing.cpu_count())

### NOTE: Preprocessing template. 

Why use Pool() and not Process()?

https://towardsdatascience.com/speed-up-your-algorithms-part-3-parallelization-4d95c0888748

In [ ]:
# import 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns

In [ ]:
# Function to crate a figure and get instance of Axis.
def axex(size):
    fig = plt.figure(figsize=(size[0],size[1])) # define plot area
    ax = fig.gca() # define axis  
    return ax

In [ ]:
loan_data = pd.read_csv('./Hackathon/Data/train.csv',low_memory=False)

In [ ]:
loan_data.describe().T

In [ ]:
loan_data.info()

In [ ]:
loan_data.head()

In [ ]:
#  Count of unique member id
len(loan_data['UniqueID'].value_counts())

In [ ]:
type(loan_data['Date.of.Birth'][0])

In [ ]:
# loan_data['loan_amnt'].isnull().sum()

# Function to find sum and percent of missing values for each column.
# Remove the one's that have no missing values.

def missing_values(df):
#     Sum null values. Then divide by the total occurrence
    sum = (df == 0).sum() 
    percent = (df == 0).sum()/len(df)*100
    missing_stats = pd.concat([sum, percent], axis=1).rename(
        columns = {
            0: 'Number',
            1: 'Percent'
        }
    )
#   drop all rows that are equal to 0.
    missing_stats = missing_stats[missing_stats.iloc[:,1] != 0]
    missing_stats.reset_index(inplace=True)
    missing_stats.sort_values('Percent', inplace=True)
    
    return missing_stats


In [ ]:
missing_values(loan_data)

In [ ]:
#  Removing columns with more than 70% of missing data.
# Get a threshold of what is 30% 
# print(loan_data.count())

# temp = [i for i in loan_data.count()<887379 *0.30]
# loan_data.drop(loan_data.columns[temp],axis=1,inplace=True)

# EDA

## Loan data vs...

In [ ]:
ax = axex([10,10])

# Average annual income plot
loan_data[['loan_default', 'Current_pincode_ID']].groupby(['Current_pincode_ID']).sum().plot(ax = ax)

## Data distribution

LTV vs distribution
ltv vs amount
LD of salaried and self_employed

In [ ]:
def univariate(df,col,vartype,hue =None):
    
    '''
    Univariate function will plot the graphs based on the parameters.
    df      : dataframe name
    col     : Column name
    vartype : variable type : continuos or categorical
                Continuos(0)   : Distribution, Violin & Boxplot will be plotted.
                Categorical(1) : Countplot will be plotted.
    hue     : It's only applicable for categorical analysis.
    
    '''
    sns.set(style="darkgrid")
    
    if vartype == 0:
        fig, ax=plt.subplots(nrows =1,ncols=3,figsize=(20,8))
        ax[0].set_title("Distribution Plot")
        sns.distplot(df[col],ax=ax[0])
        ax[1].set_title("Violin Plot")
        sns.violinplot(data =df, x=col,ax=ax[1], inner="quartile")
        ax[2].set_title("Box Plot")
        sns.boxplot(data =df, x=col,ax=ax[2],orient='v')
    
    if vartype == 1:
        temp = pd.Series(data = hue)
        print(len(temp.unique()))
        fig, ax = plt.subplots()
        width = len(df[col].unique()) + 6 + 4*len(temp.unique())
        fig.set_size_inches(width , 7)
        ax = sns.countplot(data = df, x= col, order=df[col].value_counts().index,hue = hue) 
        if len(temp.unique()) > 0:
            for p in ax.patches:
                if p.get_height() > 0:
                    ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(loan_data))), (p.get_x()+0.05, p.get_height()+20))  
        else:
            for p in ax.patches:
                ax.annotate(p.get_height(), (p.get_x()+0.32, p.get_height()+20)) 
        del temp
    else:
        exit
        
    plt.show()

In [ ]:
temp = pd.Series(data = 'hue')

### Continuous variables

In [ ]:
# dibursed_amount
univariate(df=loan_data,col='disbursed_amount',vartype=0)

In [ ]:
# loan_data.loc[(loan_data['disbursed_amount'] > 180000)]['loan_default']

loan_data.disbursed_amount.describe()

In [ ]:
loan_data.loc[(loan_data['disbursed_amount'] > 100000) &\
              (loan_data['loan_default'] == 0)]['loan_default'].count()

__Remove rows containing anomalies (amt > 1lk)? what do these anomalies mean?__

In [ ]:
univariate(df=loan_data,col='asset_cost',vartype=0)

__drop asset_cost > 1lkh?__

In [ ]:
univariate(df=loan_data,col='ltv',vartype=0)

In [ ]:
loan_data.ltv.describe()

In [ ]:
loan_data.loc[(loan_data['ltv'] < 48) & (loan_data['disbursed_amount'] > 100000)]

In [ ]:
# distribution of cns score
univariate(df=loan_data,col='PERFORM_CNS.SCORE',vartype=0)

In [ ]:
loan_data['PERFORM_CNS.SCORE'].describe()

In [ ]:
# Default and no credit rating score
x = len(loan_data.loc[(loan_data['PERFORM_CNS.SCORE'] == 0) &\
              (loan_data['loan_default'] == 1)])

y = len(loan_data.loc[(loan_data['PERFORM_CNS.SCORE'] == 0)])

# percent defaulted with no CNS score.
print(x/y)

In [ ]:
loan_data['PRIMARY.INSTAL.AMT'].head()

In [ ]:
# distribution of EMI amount
# univariate(df=loan_data,col='PRIMARY.INSTAL.AMT',vartype=0)
ax = axex([10,10])
loan_data['PRIMARY.INSTAL.AMT'].plot.hist(ax=ax)

### Categorical Variables

In [ ]:
len(loan_data)

In [ ]:
print(loan_data['loan_default'].value_counts())
print(50611/182543)

In [ ]:
# Employment type count plot
univariate(df=loan_data,col='Employment.Type',vartype=1)

In [ ]:
univariate(df=loan_data[(loan_data['Employment.Type'] == 'Salaried')],col='loan_default',vartype=1)

In [ ]:
19910/77948

In [ ]:
univariate(df=loan_data[(loan_data['Employment.Type'] == 'Self employed')],col='loan_default',vartype=1)

In [ ]:
29057/98578

In [ ]:
# Count of loan applicant from each state
univariate(df=loan_data,col='State_ID',vartype=1)

In [ ]:
%matplotlib inline
# LD vs Sate
univariate(df=loan_data,col='State_ID',vartype=1, hue='loan_default')


In [ ]:
# Count of instances per manufacturer
univariate(df=loan_data,col='manufacturer_id',vartype=1)

In [ ]:
len(loan_data.loc[(loan_data['manufacturer_id'] == 152) |\
              (loan_data['manufacturer_id'] == 153) |\
              (loan_data['manufacturer_id'] == 156)
             ])

Drop manu_id 152, 153, 156 ? total == 19 

In [ ]:
loan_data['manufacturer_id'].value_counts().index

In [ ]:
loan_data.loc[(loan_data['manufacturer_id'] == 153) & (loan_data['loan_default']==1)]['loan_default']


In [ ]:
# --------ANAAAALYSISSSSSSSSS-------------
# fig, ax = plt.subplots()
# fig.set_size_inches(20 ,20 )
# ax = sns.countplot(data = loan_data, x= 'manufacturer_id', order=loan_data['manufacturer_id'].\
#                    value_counts().index,hue = 'loan_default')
# for p in ax.patches:
# #     
#     if p.get_height() > 0:
#         print(p.get_height())
#         ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(loan_data))),\
#                     (p.get_x()+0.05, p.get_height()+20))

In [ ]:
# LD vs manufacturer

'''
    152 and 156 have no defaults. p.get_height() is returning a nan value if value = 0.
    153 has 4 loan_defaults. percentage base is entire df len. therefore 0% for 153 comes 
    after flooring the percent value.
    '''
univariate(df=loan_data,col='manufacturer_id',vartype=1, hue='loan_default')


In [ ]:
# # Count of instances per branch_id
# univariate(df=loan_data,col='branch_id',vartype=1)

# LD grouped by branchid. 

ax = axex([15,10])

loan_data[['loan_default', 'branch_id']].\
    groupby('branch_id').\
    count().\
    plot.bar(ax=ax)


ax.set_title('LD vs Branch')
ax.set_xlabel('Branch')
ax.set_ylabel('LD')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20 ,20)
ax = sns.countplot(data = loan_data, x= 'branch_id', order=loan_data['branch_id'].\
                   value_counts().index,hue = 'loan_default')
for p in ax.patches:
#     
    if p.get_height() > 0:
        ax.annotate('{:1.1f}'.format((p.get_height())),\
                    (p.get_x()+0.05, p.get_height()+20))

## Bivariate plots 
__relation of 2 features to the target(categorical type)__ Used to check correlation of the features.

In [ ]:
sns.relplot(x="asset_cost", y="disbursed_amount", hue="loan_default", data=loan_data);

__Probably confirms removing val > 1lkh based on disbursed_amount and asset_cost distribution__

In [ ]:
sns.catplot(x="loan_default", y="disbursed_amount", data=loan_data);

In [ ]:
plt.figure(figsize=(16,12))
sns.relplot(data =loan_data, x='PRIMARY.INSTAL.AMT', y='disbursed_amount', hue ='loan_default')
plt.title('EMI vs Disbursed Amount')
plt.show()

### Correlation Map

In [ ]:
# loan_correlation = loan_data.corr()
# loan_correlation

In [ ]:
# f, ax = plt.subplots(figsize=(14, 9))
# sns.heatmap(loan_correlation, 
#             xticklabels=loan_correlation.columns.values,
#             yticklabels=loan_correlation.columns.values,annot= True)
# plt.show()

# Feature Engineering

### Changing duration to number of months

In [ ]:
loan_data['AVERAGE.ACCT.AGE'].head()

In [ ]:
import re

def pat1(x):
    result = re.search('([0-9]*) [0-9]*', x)
    return int(result.group(1))

def pat2(x):
    result = re.search('[0-9]* ([0-9]*)', x)
    return int(result.group(1))
    
def change_to_months(loan_data):
    loan_data['avg_tenure'] = loan_data['AVERAGE.ACCT.AGE'].str.replace(r'[a-zA-Z]*', '')
    loan_data['cred_hist_len'] = loan_data['CREDIT.HISTORY.LENGTH'].str.replace(r'[a-zA-Z]*', '')
    loan_data['avg_tenure'] = loan_data['avg_tenure'].apply(lambda x: pat1(x)*12 + pat2(x))
    loan_data['cred_hist_len'] = loan_data['cred_hist_len'].apply(lambda x: pat1(x)*12 + pat2(x))
    


### Create age from DOB

In [ ]:
def change_to_age(loan_data):
    now = pd.Timestamp('now')
    loan_data['Date.of.Birth'] = pd.to_datetime(loan_data['Date.of.Birth'])
    loan_data['Date.of.Birth'] = loan_data['Date.of.Birth'].where(loan_data['Date.of.Birth'] < now, loan_data['Date.of.Birth'] -  np.timedelta64(100, 'Y'))
    loan_data['age'] = (now - loan_data['Date.of.Birth']).astype('<m8[Y]')

### Convert employment type to numerical format

In [ ]:
def change_to_binary(loan_data):
    # 1 - Salaried
    # 0 - Self-employed
    loan_data['Employment.Type'] = loan_data['Employment.Type'].map({'Salaried': 1, 'Self employed' : 0})
    loan_data.fillna(value=-1, inplace=True)

### Converting all flags to one feature

In [ ]:
def create_traceability(loan_data):
    loan_data['traceability'] = (loan_data['MobileNo_Avl_Flag'] + loan_data['Aadhar_flag'] + loan_data['PAN_flag'] + loan_data['VoterID_flag'] + loan_data['Driving_flag'] + loan_data['Passport_flag']) / 6.0

Should the flags be dropped? Test with model

### Dropping entries of asset_cost > 100000

In [ ]:
def drop_asset_cost_outliers(df):
    df = df.drop(df[df['disbursed_amount'] > 100000].index)

### Call functions on datasets

In [ ]:
test = pd.read_csv('./Hackathon/Data/test.csv')

change_to_months(loan_data)
change_to_age(loan_data)
change_to_binary(loan_data)
create_traceability(loan_data)
drop_asset_cost_outliers(loan_data)

change_to_months(test)
change_to_age(test)
change_to_binary(test)
create_traceability(test)


In [ ]:
print(test.isnull().sum())
print("-----------")
print(loan_data.isnull().sum())

In [ ]:
loan_data['traceability'].value_counts()
univariate(df=loan_data, col='traceability', vartype=1, hue='loan_default')

# Model fitting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
# from sklearn import metrics, cross_validation
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [ ]:
df_model = loan_data.copy(deep=True)
df_test_model = test.copy(deep=True)

In [ ]:
df_model.drop([
    'UniqueID',
    'branch_id',
    'supplier_id',
    'manufacturer_id',
    'Current_pincode_ID',
    'VoterID_flag',
    'Aadhar_flag'.
    'MobileNo_Avl_Flag',
    'PAN_flag',
    'VoterID_flag',
    'Driving_flag',
    'Passport_flag',
    'Date.of.Birth',
    'AVERAGE.ACCT.AGE',
    'CREDIT.HISTORY.LENGTH',
    'PERFORM_CNS.SCORE.DESCRIPTION',
    'Employee_code_ID',
    'State_ID',
    'DisbursalDate'
], axis=1, inplace=True)

df_test_model.drop([
    'UniqueID',
    'branch_id',
    'supplier_id',
    'manufacturer_id',
    'Current_pincode_ID',
    'VoterID_flag',
    'Aadhar_flag',
    'MobileNo_Avl_Flag',
    'PAN_flag',
    'VoterID_flag',
    'Driving_flag',
    'Passport_flag',
    'Date.of.Birth',
    'AVERAGE.ACCT.AGE',
    'CREDIT.HISTORY.LENGTH',
    'PERFORM_CNS.SCORE.DESCRIPTION',
    'Employee_code_ID',
    'State_ID',
    'DisbursalDate'
], axis=1, inplace=True)


In [ ]:
df_model.rename(index=str, columns={"loan_default": "Target"}, inplace=True)


### Balancing the training dataset as 50-50

In [ ]:
def find_rmse(reg): 
    #   Train test split  
#     X_train, X_test, y_train, y_test = train_test_split(df_model.drop('Target',axis=1),df_model['Target'],test_size=0.20, stratify=y, random_state=154)
    X_train = df_model.drop('Target', axis=1)
    y_train = df_model['Target']
    X_test = df_test_model.copy(deep=True)
    X_train.index = X_train.index.astype('int64')
    y_train.index = y_train.index.astype('int64')
    X_test.index = X_test.index.astype('int64')
    # Balancing the dataset
    
#     np_y_train = y_train.values
#     np_X_train = X_train.values
#     total_num_of_ones = int(np.sum(y_train))
#     # print(total_num_of_ones)
#     zero_counter = 0
#     indices_to_remove = []

#     for i in range(np_y_train.shape[0]):
#         if np_y_train[i] == 0:
#             if zero_counter < total_num_of_ones:
#                 zero_counter += 1
#             else:
#                 indices_to_remove.append(i)

#     X_train = np.delete(np_X_train, indices_to_remove, axis=0)
#     y_train = np.delete(np_y_train, indices_to_remove, axis=0)

#     print(len(X_train))
#     print(len(y_train))
    
    # Normalising the dataset     
#     sc = StandardScaler()
#     X_train = sc.fit_transform(X_train)
#     X_test=sc.transform(X_test)
    
#     # Fit the model
#     reg.fit(X_train, y_train)
    
#     res = cross_val_score(reg, X_train, y_train, cv=10, scoring='accuracy')
#     print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
    
#     y_out = reg.predict(X_test)
# #     print(len(y_out))

    # Implementing XGBoost and stratified k-fold
    
    # Converting X_train and y_train to a DF
#     X_train = pd.DataFrame(X_train)
#     y_train = pd.DataFrame(y_train)
    
    param_grid = [{'min_child_weight': np.arange(0.1, 10.1, 0.1)}]
    i=1
    kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
    for train_index,test_index in kf.split(X_train,y_train):
        print('\n{} of kfold {}'.format(i,kf.n_splits))
        xtr,xvl = X_train.loc[train_index],X_train.loc[test_index]
        ytr,yvl = y_train[train_index],y_train[test_index]
        model = GridSearchCV(XGBClassifier(n_estimators=10, n_jobs=4), param_grid, cv=10, scoring= 'f1',iid=True)
        model.fit(xtr, ytr)
        print (model.best_params_)
        pred=model.predict(X_test)
        print('accuracy_score',accuracy_score(yvl,pred))
        i+=1
    
    y_out = predict(X_test)
    op_df = pd.DataFrame({'UniqueID': test['UniqueID'], 'loan_default': y_out})
    print(op_df['loan_default'].value_counts())
    op_df.to_csv(path_or_buf='./rand_for.csv', index=False, index_label='UniqueID')
#     print('Accuracy score: ')
#     print(reg.score(X_test, y_test))

#     error = np.sqrt(mean_squared_error(y_test, y_out))
                                
#     print(str(error)+ repr(reg)[0:4]) #using repr to get initials of the name of the classifier that this run used

In [ ]:
from multiprocessing import Pool

#Add imported classes of classification models to be used.
regressors_list = [LogisticRegression(), RandomForestClassifier()] 
pool = Pool(1)
pool.map(find_rmse,regressors_list)
pool.close()
pool.join()